In [4]:
import sys
from env_key_manager import APIKeyManager

# 创建实例
key_manager = APIKeyManager()

# 设置环境变量
key_manager.setup_api_key(["OPENAI_API_KEY", "OPENAI_BASE_URL"])

# 查看Python版本
!python -V
# 查看安装的库
if 'win' in sys.platform.lower():
    !pip list | findstr "lang openai llm tiktoken chromadb cryptography duck unstructured numpy scipy"
else:
    !pip list | grep -E 'lang|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

Python 3.10.16
cryptography                      44.0.2
duckduckgo_search                 6.3.7
langchain                         0.3.19
langchain-community               0.3.18
langchain-core                    0.3.49
langchain-deepseek                0.1.3
langchain-openai                  0.3.11
langchain-text-splitters          0.3.6
langgraph                         0.3.21
langgraph-checkpoint              2.0.23
langgraph-prebuilt                0.1.7
langgraph-sdk                     0.1.60
langserve                         0.3.1
langsmith                         0.3.8
numpy                             1.26.4
openai                            1.69.0
scipy                             1.15.2
tiktoken                          0.9.0


# 构建语义搜索引擎

本教程将帮助您熟悉LangChain的[文档加载器](/docs/concepts/document_loaders)、[嵌入模型](/docs/concepts/embedding_models)和[向量存储](/docs/concepts/vectorstores)抽象概念。这些抽象设计旨在支持从（向量）数据库及其他数据源检索信息，以便与LLM工作流集成。对于需要在模型推理过程中获取数据进行分析的应用（例如检索增强生成即[RAG](/docs/concepts/rag)，参见我们的[RAG教程](/docs/tutorials/rag)），这些组件尤为重要。

在这里，我们将构建一个基于PDF文档的搜索引擎。这将使我们能够检索与输入查询相似的PDF文档段落。

## 概念

本指南重点介绍文本数据的检索。我们将涵盖以下概念：

- 文档与文档加载器；
- 文本分割器；
- 嵌入；
- 向量存储与检索器。

## 安装设置

### Jupyter 笔记本

本教程及其他教程或许在 Jupyter notebook 中运行最为便捷。有关安装说明，请参阅[此处](https://jupyter.org/install)。

### 安装

本教程需要安装 `langchain-community` 和 `pypdf` 这两个包：

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
<CodeBlock language="bash">pip install langchain-community pypdf</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
<CodeBlock language="bash">conda install langchain-community pypdf -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>


更多详情，请参阅我们的[安装指南](/docs/how_to/installation)。

### LangSmith

使用LangChain构建的许多应用程序将包含多个步骤，涉及多次LLM调用。
随着这些应用变得越来越复杂，能够检查链或代理内部的具体运行情况变得至关重要。
最佳方式是使用 [LangSmith](https://smith.langchain.com)。

在以上链接完成注册后，请确保设置环境变量以开始记录追踪数据：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```
```python
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```
## 文档与文档加载器

LangChain 实现了 [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) 抽象类，用于表示文本单元及其关联元数据。该类包含三个属性：

- `page_content`: 一个表示内容的字符串；
- `metadata`: 一个包含任意元数据的字典；
- `id`: （可选）文档的字符串标识符。

`metadata` 属性可以捕获文档来源、与其他文档的关系以及其他相关信息。请注意，单个 `Document` 对象通常代表较大文档的一个片段。

我们可以根据需要生成示例文档：
```python
from langchain_core.documents import Document

doc = [
  Document(
  page_content="狗是极好的伙伴，以其忠诚和友好而闻名。"
  metadata={"source": "哺乳动物宠物文档"},
  ),
  Document(
  page_content="猫是独立的宠物，通常喜欢拥有自己的空间。"
  metadata={"source": "哺乳动物宠物文档"},
  ),
]
```

然而，LangChain生态系统实现了[文档加载器](/docs/concepts/document_loaders)，这些加载器[与数百种常见数据源集成](/docs/integrations/document_loaders/)。这使得将这些来源的数据整合到您的AI应用中变得轻而易举。

### 加载文档

让我们将PDF文件加载为一系列`Document`对象。LangChain代码库中有一个示例PDF文件[在此处](https://github.com/langchain-ai/langchain/tree/master/docs/docs/example_data)——这是耐克公司2023年的10-K年报文件。我们可以查阅LangChain文档中[可用的PDF文档加载器](/docs/integrations/document_loaders/#pdfs)。这里我们选择相对轻量级的[PyPDFLoader](/docs/integrations/document_loaders/pypdfloader/)。

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../example_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


:::提示

请参阅[此指南](/docs/how_to/document_loader_pdf/)获取关于PDF文档加载器的更多详细信息。

:::

`PyPDFLoader` 为每个 PDF 页面加载一个 `Document` 对象。对于每个对象，我们可以轻松访问：

- 页面的字符串内容；
- 包含文件名和页码的元数据。

In [5]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


### 分割

无论是出于信息检索还是下游问答的目的，将整个页面作为处理单元可能过于粗糙。我们的最终目标是检索出能回答输入查询的`文档`对象，因此对PDF进行进一步拆分将有助于确保文档相关部分的意义不会被周围文本"稀释"。

我们可以使用[文本分割器](/docs/concepts/text_splitters)来实现这一目的。这里我们将使用一个基于字符分割的简单文本分割器，将文档分割成每块1000个字符的片段。
以200个字符的块间重叠。这种重叠有助于
降低将声明与重要内容分离的可能性
与上下文相关的信息。我们使用
[递归字符文本分割器](/docs/how_to/recursive_text_splitter)
这将递归地使用常见分隔符来分割文档
在确保每个数据块大小合适之前持续添加新行。这是
适用于通用文本场景的推荐文本分割器。

我们设置 `add_start_index=True` 以便记录每个字符的起始索引位置
分割文档起始于初始文档内部被保留
元数据属性“start_index”。

请参阅[本指南](/docs/how_to/document_loader_pdf/)获取有关处理PDF的更多详细信息，包括如何从特定章节和图像中提取文本。

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

## 嵌入

向量搜索是一种存储和检索非结构化数据（如非结构化文本）的常用方法。其核心思想是存储与文本关联的数值向量。给定查询时，我们可以将其[嵌入](/docs/concepts/embedding_models)为相同维度的向量，并通过向量相似度度量（如余弦相似度）来识别相关文本。

LangChain 支持来自[数十家供应商](/docs/integrations/text_embedding/)的嵌入模型。这些模型定义了如何将文本转化为数值向量。让我们选择一个模型：

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs customVarName="embeddings" />

In [6]:
# | output: false
# | echo: false

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

有了生成文本嵌入的模型后，我们接下来可以将它们存储在一个支持高效相似性搜索的特殊数据结构中。

## 向量存储

LangChain [VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 对象包含向存储中添加文本和 `Document` 对象的方法，以及使用各种相似性度量进行查询的功能。它们通常使用[嵌入](/docs/how_to/embed_text)模型进行初始化，这些模型决定了如何将文本数据转换为数值向量。

LangChain 包含一系列与不同向量存储技术的[集成方案](/docs/integrations/vectorstores)。部分向量存储由服务提供商托管（例如各类云服务商）并需要特定凭证才能使用；另一些（如[Postgres向量存储](/docs/integrations/vectorstores/pgvector)）运行在可本地部署或通过第三方管理的独立基础设施中；还有些支持轻量级任务的纯内存运行模式。现在让我们选择一个向量存储：

import VectorStoreTabs from "@theme/VectorStoreTabs";

<VectorStoreTabs/>

In [15]:
# | output: false
# | echo: false

from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

在实例化我们的向量存储之后，我们现在可以对文档进行索引了。

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

请注意，大多数向量存储实现都允许您连接到现有的向量存储——例如，通过提供客户端、索引名称或其他信息。有关更多详细信息，请参阅特定[集成](/docs/integrations/vectorstores)的文档。

一旦我们实例化了一个包含文档的 `VectorStore`，就可以对其进行查询。[VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 包含了以下查询方法：
- 同步与异步；
- 通过字符串查询和向量查询；
- 包含和不包含返回相似度分数；
- 通过相似度和[最大边际相关性](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search)（在检索结果中平衡查询相似性与多样性）。

这些方法通常会在其输出中包含一个[Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document)对象的列表。

### 使用方法

嵌入通常将文本表示为“密集”向量，使得含义相似的文本在几何上接近。这样我们只需输入问题即可检索相关信息，而无需了解文档中使用的任何特定关键词。

根据与字符串查询的相似性返回文档：

In [17]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

APIConnectionError: Connection error.

异步查询：

In [18]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

APIConnectionError: Connection error.

返回分数：

In [11]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.23699893057346344

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS
The following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
•NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
•NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

根据与嵌入查询的相似性返回文档：

In [11]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
•Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
•Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
•Unfavorable changes in net foreign currency exchange rates, including hedges; and
•Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:' met

了解更多：

- [API 参考文档](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html)
- [操作指南](/docs/how_to/vectorstores)
- [集成相关文档](/docs/integrations/vectorstores)

## 检索器

LangChain 的 `VectorStore` 对象并未继承 [Runnable](https://python.langchain.com/api_reference/core/index.html#langchain-core-runnables)。而 LangChain 的 [Retrievers](https://python.langchain.com/api_reference/core/index.html#langchain-core-retrievers) 属于 Runnable 类型，因此它们实现了一套标准方法（如同步/异步的 `invoke` 和 `batch` 操作）。虽然我们可以基于向量存储构建检索器，但检索器同样能够与非向量存储的数据源（例如外部 API）进行交互。

我们可以自行创建一个简化版本，无需继承`Retriever`类。只需选定检索文档的方法，就能轻松构建一个可执行流程。下面我们将围绕`similarity_search`方法进行实现：

In [14]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(metadata={'page': 4, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.\n2023 FORM 10-K 2')],
 [Document(metadata={'page': 3, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 0}, page_content='Table of Contents\nPART I\nITEM 1. BUSINESS\nGENERAL\nNIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"\n"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectivel

向量存储库实现了`as_retriever`方法，该方法会生成一个检索器，具体来说是[VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html)。这些检索器包含特定的`search_type`和`search_kwargs`属性，用于标识底层向量存储库要调用的方法以及如何对其进行参数化。例如，我们可以通过以下方式复制上述内容：

In [13]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(metadata={'page': 4, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.\n2023 FORM 10-K 2')],
 [Document(metadata={'page': 3, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 0}, page_content='Table of Contents\nPART I\nITEM 1. BUSINESS\nGENERAL\nNIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"\n"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectivel

`VectorStoreRetriever` 支持三种检索类型：默认的 `"similarity"`（相似度检索）、`"mmr"`（最大边际相关性检索，如上所述）以及 `"similarity_score_threshold"`（相似度分数阈值检索）。最后一种方式可用于根据相似度分数对检索器输出的文档进行阈值筛选。

检索器可以轻松集成到更复杂的应用程序中，例如[检索增强生成（RAG）](/docs/concepts/rag)应用。这类应用将给定问题与检索到的上下文结合，生成面向大语言模型（LLM）的提示词。要了解更多关于构建此类应用的信息，请参阅[RAG教程](/docs/tutorials/rag)。

### 了解更多：

检索策略可以丰富而复杂。例如：

- 我们可以从查询中[推断出严格的规则和筛选条件](/docs/how_to/self_query/)（例如，“使用2020年后发布的文档”）；
- 我们可以[返回以某种方式链接到检索上下文的文档](/docs/how_to/parent_document_retriever/)（例如，通过某些文档分类法）；
- 我们可以为每个上下文单元生成[多重嵌入](/docs/how_to/multi_vector)；
- 我们可以[集成多个检索器的结果](/docs/how_to/ensemble_retriever)；
- 我们可以为文档分配权重，例如，赋予[近期文档](/docs/how_to/time_weighted_vectorstore/)更高的权重。

[检索器](/docs/how_to#retrievers)部分的操作指南涵盖了这些以及其他内置检索策略。

扩展 [BaseRetriever](https://python.langchain.com/api_reference/core/retrievers/langchain_core.retrievers.BaseRetriever.html) 类来实现自定义检索器也非常简单。具体操作指南请参阅[此处](/docs/how_to/custom_retriever)。


## 后续步骤

你现在已经了解了如何基于PDF文档构建语义搜索引擎。

有关文档加载器的更多信息：

- [概念指南](/docs/concepts/document_loaders)
- [操作指南](/docs/how_to/#document-loaders)
- [可用集成](/docs/integrations/document_loaders/)

有关嵌入的更多信息：

- [概念指南](/docs/concepts/embedding_models/)
- [操作指南](/docs/how_to/#embedding-models)
- [可用集成](/docs/integrations/text_embedding/)

有关向量存储的更多信息：

- [概念指南](/docs/concepts/vectorstores/)
- [操作指南](/docs/how_to/#vector-stores)
- [可用集成](/docs/integrations/vectorstores/)

有关RAG的更多信息，请参阅：

- [构建检索增强生成（RAG）应用](/docs/tutorials/rag/)
- [相关操作指南](/docs/how_to/#qa-with-rag)